<a href="https://colab.research.google.com/github/ofekih/LightsOutChallenge/blob/main/LightsOutChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!git clone https://github.com/ofekih/LightsOutChallenge.git
!rm -r .git/ LightsOutChallenge/LightsOutChallenge.ipynb
!mv LightsOutChallenge/* LightsOutChallenge/.??* .
!rmdir LightsOutChallenge/

Cloning into 'LightsOutChallenge'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 14 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (14/14), done.


In [24]:
!cat README.md

# LightsOutChallenge
CS169 Group Project


This will do $5+2$

In [28]:
with open('README.md', 'r') as file:
    print(file.readlines())

['# LightsOutChallenge\n', 'CS169 Group Project\n']
